In [1]:
import pandas as pd
df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [2]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [3]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [8]:
dfnew = pd.get_dummies(df, columns=['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope'], drop_first=True).astype('int')
dfnew.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0,0,1,0,1,0,1,0,0,0,1


In [9]:
dfnew.size

14688

In [10]:
from scipy.stats import zscore
dfnorm = dfnew[(zscore(dfnew) < 3).all(axis=1)]
dfnorm.size

13600

In [11]:
dfnorm.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
count,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.0,850.000000,850.000000,850.000000,850.000000,850.000000
mean,53.342353,131.583529,196.295294,0.230588,136.343529,0.662353,0.550588,0.790588,0.201176,0.236471,0.0,0.612941,0.192941,0.412941,0.503529,0.437647
std,9.303759,17.660740,108.801853,0.421457,25.433339,0.891060,0.497727,0.407128,0.401116,0.425165,0.0,0.487364,0.394840,0.492652,0.500282,0.496389
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.000000,120.000000,171.000000,0.000000,119.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,54.000000,130.000000,221.000000,0.000000,138.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,1.000000,0.000000
75%,60.000000,140.000000,266.000000,0.000000,155.750000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,1.000000,1.000000,1.000000
max,77.000000,185.000000,518.000000,1.000000,202.000000,3.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dfnorm.drop(columns=['HeartDisease']))

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, dfnorm['HeartDisease'], test_size=0.3)

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

cross_val_score(SVC(), X_train, y_train, cv=5).mean()

np.float64(0.8789915966386553)

In [20]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
)

cross_val_score(bagging, X_train, y_train, cv=5).mean()

np.float64(0.8840336134453782)

In [21]:
from sklearn.tree import DecisionTreeClassifier
cross_val_score(DecisionTreeClassifier(), X_train, y_train, cv=5).mean()

np.float64(0.8033613445378152)

In [22]:
baggingD = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
)

cross_val_score(baggingD, X_train, y_train, cv=5).mean()

np.float64(0.8605042016806722)

In [33]:
from sklearn.ensemble import RandomForestClassifier

cross_val_score(RandomForestClassifier(), X_train, y_train, cv=5).mean()

np.float64(0.8857142857142858)